# Gene set and pathway

![](./images/Module3/Workflow3.jpg)

Differential expression (DE) analysis typically yields a list of genes or proteins that require further interpretation.
Our intention is to use such lists to gain novel insights about genes and proteins that may have roles in a given
phenomenon, phenotype or disease progression. However, in many cases, gene lists generated from DE analysis are
difficult to interpret due to their large size and lack of useful annotations. Hence, pathway analysis (also known
as gene set analysis or over-representation analysis), aims to reduce the complexity of interpreting gene lists
via mapping the listed genes to known (i.e. annotated) biological pathways, processes and functions.
This learning module introduces common curated biological databases including Gene Ontology (GO),
Kyoto Encyclopedia of Genes and Genomes (KEGG).

## Learning Objectives:
1. Introduction to Ontology and Gene Ontology.
2. Introduction to KEGG Pathway Database.
3. Downloading terms, pathway gene set from GO and KEGG.
4. Saving results to GMT file format.

## Ontology and Gene Ontology
### Overview
In this section we will learn about the concept of gene ontology in Bioinformatics. Ontology is a specification
of a conceptualization which is a set of concepts within a domain, defined by a shared vocabulary to
denote the types and properties of the concepts as well as the relationships between the concepts.
Ontology plays an important role in the field of bioinformatics. Ontology enable communicate unambiguously e.g.,
to understand across different groups’ annotations of various genomes. Also, it allows the knowledge to be represented
in a computable and structured to perform automated analyses by computer programs.

The Gene Ontology (GO) defines a structured, common, controlled vocabulary to describe attributes of genes and gene products
across organisms. Collaboration is key to build a consensus vocabulary. But the term gene ontology, or GO, is commonly used
to refer to both, which is sometimes a source of potential confusion. In order to avoid this, here we will use the term “GO”
to describe the set of terms and their hierarchical structure and “GO annotations” to describe the set of associations between
genes and GO terms. The GO is divided into three categories to describe the genes and gene products from three different
angles: Molecular Function, Biological Process, and Cellular Component.

The structure of GO can be described in terms of into directed acyclic graphs (DAGs), where each GO term is a node,
and the relationships between the terms are edges between the nodes. GO is loosely hierarchical, with ‘child’ terms
being more specialized than their ‘parent’ terms, but unlike a strict hierarchy, a term may have more than one parent
term (note that the parent/child model does not hold true for all types of relations). The structure of the controlled
vocabularies are intended to reflect true, biological relationships. In contrast to strict hierarchies, DAGS allow
multiple relationships between a more granular (child) term and a more general parent term. The relationship between
terms affects how queries are made. For example,a query for all genes with binding activity would include transcription
factors as well as genes with other types of binding activity (such as protein binding, ligand binding). The illustration
of category and structure of GO is shown in the figure below:

![](./images/Module3/GO_Structure.jpg)
*(Source: https://www.ebi.ac.uk/, http://geneontology.org/)*

### Gene ontology relationship
In DAGs graph, *terms* are represented as *nodes* and *relations* (also known as *object properties*) between the *terms*
are *edges*. There are commonly used relationships in GO such as *is a* (is a *subtype of*), *part of, has part, regulates,
negatively regulates and positively regulates*. All terms (except from the root terms representing each aspect) have an
is a sub-class relationship to another terms.

Examples:

```{admonition} *is a* relation
**GO:1904659:glucose transport** *is* a **GO:0015749:monosaccharide transport**.

The *is a* relation forms the basic structure of GO. If we say A *is a* B, we mean that node A is a subtype of node B

```

```{admonition} *is part of* relation
**GO:0031966:mitochondrial membrane** *is part of* **GO:0005740:mitochondrial envelope**

The *part of* relation is used to represent part-whole relationships. A *part of* relation would only be added between
A and B if B is **necessarily** *part of* A: wherever B exists, it is as *part of* A, and the presence of the B implies
the presence of A. However, given the occurrence of A, we cannot say for certain that B exists.
```

```{admonition} *regulates* relation
**GO:0098689:latency-replication decision** *regulates* **GO:0019046:release from viral latency**

A relation that describes case in which one process directly affects the manifestation of another process or quality,
i.e. the former *regulates* the latter.
```

A more specific case with more nodes and edges can be seen at the figure below:
<br>
![](./images/Module3/GO_Relation.jpg)
*(Source: https://advaitabio.com/)* <br>
For more technical information about relations and their properties used in GO and other ontologies see the
<a href="https://obofoundry.org/ontology/ro.html">OBO Relations Ontology (RO)</a>


### GO storage file formats
GO terms are updated monthly in the following formats:
* OBO 1.4 files are human-readable (in addition to machine-readable) and can be opened in any text editor.
* OWL files can be read by <a href="https://protege.stanford.edu/">Protégé</a> text editor.

 In this learning module, we will only use ".OBO" to obtain GO terms.The OBO file format is for representing ontologies and controlled vocabularies. The format itself attempts to achieve the following goals:
 * Human readability
 * Ease of parsing
 * Extensibility
 * Minimal redundancy

The file structure can is shown in the following figure.


![](./images/Module3/OBO_Format.jpg)

The OBO file has a header which is an unlabeled section at the beginning of the document. The header ends when the first term is encountered. Next, term is represented in labeled section with the tag *[Term]*. Under each term, we can find other information such as term ID, official name, category (namcespace), term definition, synonym and relation to other GO term.

At this step, we still don't know what genes are related to which GO terms. In order to retrieve custom sets of gene ontology annotations for any list of genes from organisms, NCBI has published a Gene2GO database that obtain GO terms and the entrez gene ids related to those go terms. The database can be retrieve from <a href="https://ftp.ncbi.nih.gov/gene/DATA/gene2go.gz">here</a> text editor. The Gene2GO database can be viewed using text editor, the file structure is presented in the figure below:

![](./images/Module3/Gene2GO.jpg)

The OBO and Gene2GO databases will be used in combination to obtain GO term and related genes for enrichment analysis.

### Retrieving GO terms from DE gene list
This section focuses on downloading related GO terms based on the DE genelist obtained from the DE analysis in the previous section.
Here, we will use `toGO` and `hgu133plus2.db` R packages to obtain GO terms. The `topGO` package has built-in functions that use Gene2GO databases to retrieve GO terms from the gene ID give by DE analysis. Since the dataset we used in the Module 02 was generated for human, we will use `hgu133plus2.db` database to map probeID to gene symbols.
The installation process of two package can be done by the script below:

In [1]:
# Installation of topGO and hgu133plus2.db package
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("topGO", update = F))
  suppressWarnings(BiocManager::install("hgu133plus2.db", update = F))
  suppressWarnings(BiocManager::install("AnnotationDbi", update = F))
})

package 'topGO' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\bangt\AppData\Local\Temp\Rtmp4Wsmf1\downloaded_packages
package 'AnnotationDbi' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\bangt\AppData\Local\Temp\Rtmp4Wsmf1\downloaded_packages


In [3]:
# Loading the library
suppressPackageStartupMessages({
  suppressWarnings(library("topGO"))
  suppressWarnings(library("hgu133plus2.db"))
  suppressWarnings(library("AnnotationDbi"))
})

Load the DE genelist generated from the DE analysis using `limma`.

In [4]:
# Loading the DE result
data = readRDS("./data/DE_genes.rds")

By default, the DE analysis performed by `limma` contains multiple features. However, adjusted *p-value* and gene ID are the most important features for enrichment analysis. We can use the following code to list of gene IDs and their *p-value*.

In [5]:
# Get p-value from DE results
genelist <- data$adj.P.Val
# Assgin gene IDs to associated p-values
names(genelist) <- data$PROBEID

After successfully obtaining the genelist, we need to map the gene IDs to the gene symbols using `hgu133plus2.db`.

In [8]:
# Map gene IDs to gene symbols
gene <- suppressMessages(AnnotationDbi::select(hgu133plus2.db, names(genelist), "SYMBOL"))

In [9]:
# Remove duplicated gene IDs
gene <- gene[!duplicated(gene[,1]),]

# Assign result to a new genlist with gene symbols
geneList2 <- genelist
names(geneList2) <- gene[,2]

Now, we can search for related GO terms based on the new gene list using `topGO` package. First, we need to create a `topGOdata`
object.

In [10]:
# Retrieve all the GO terms related to the genelist obtained from the expression matrix
GOdata <- new("topGOdata", description = "",ontology = "BP",
                    allGenes = geneList2, geneSel = function(x)x, nodeSize = 10,
                    annot = annFUN.org, ID = "alias", mapping = "org.Hs.eg")


Building most specific GOs .....

	( 12292 GO terms found. )


Build GO DAG topology ..........

	( 15812 GO terms and 36000 relations. )


Annotating nodes ...............

	( 16261 genes annotated to the GO terms. )



We can search for related GO terms using `geneInTerm` function and view the term with associated genes.

In [11]:
# Obtain a list of genes for each GO term
allGO = genesInTerm(GOdata)
allGO[1:5]

$`GO:0000002`
 [1] "AKT3"     "DNA2"     "DNAJA3"   "ENDOG"    "FLCN"     "LIG3"    
 [7] "LONP1"    "MDP1"     "MEF2A"    "METTL4"   "MGME1"    "MPV17"   
[13] "OPA1"     "PARP1"    "PIF1"     "PIM1"     "POLB"     "POLG"    
[19] "POLG2"    "PPARGC1A" "PRIMPOL"  "RRM1"     "RRM2B"    "SESN2"   
[25] "SLC25A33" "SLC25A36" "SLC25A4"  "SSBP1"    "STOML2"   "STOX1"   
[31] "TOP3A"    "TP53"     "TWNK"     "TYMP"    

$`GO:0000003`
   [1] "A1CF"      "A2M"       "AAAS"      "ABAT"      "ABCC8"     "ABHD2"    
   [7] "ACE"       "ACE2"      "ACOD1"     "ACOX1"     "ACR"       "ACRBP"    
  [13] "ACSBG1"    "ACSBG2"    "ACSL4"     "ACTL7A"    "ACTL9"     "ACTR2"    
  [19] "ACTR3"     "ACVR1"     "ACVR1B"    "ACVR1C"    "ACVR2A"    "ADA"      
  [25] "ADAD1"     "ADAM15"    "ADAM18"    "ADAM19"    "ADAM2"     "ADAM20"   
  [31] "ADAM21"    "ADAM28"    "ADAM29"    "ADAM32"    "ADAM5"     "ADAMTS1"  
  [37] "ADAMTS16"  "ADAMTS2"   "ADCY10"    "ADCY3"     "ADCY7"     "ADCYAP1"  
  [43] "ADCYAP1R1" "ADGRG1"    "ADGRG2"    "ADIG"      "ADM"       "ADNP"     
  [49] "ADRA2A"    "ADRA2B"    "AFF4"      "AFP"       "AGFG1"     "AGO2"     
  [55] "AGO4"      "AGRP"      "AGT"       "AKAP3"     "AKAP4"     "AKR1C3"   
  [61] "AKT1"      "ALDOA"     "ALKBH1"    "ALKBH5"    "ALOX15B"   "ALPL"     
  [67] "AMBP"      "AMD1"      "AMH"       "AMHR2"     "ANG"       "ANGPT2"   
  [73] "ANKRD31"   "ANKRD49"   "ANTXR1"    "ANXA1"     "AP3B1"     "APELA"    
  [79] "APOB"      "APOL2"     "APOL3"     "APP"       "AQP4"      "AR"       
  [85] "AREG"      "ARG1"      "ARHGDIB"   "ARID4A"    "ARID4B"    "ARID5B"   
  [91] "ARMC12"    "ARMC2"     "ARNT"      "ARNTL"     "ARRB2"     "ASB1"     
  [97] "ASCL2"     "ASF1B"     "ASH1L"     "ASPM"      "ASZ1"      "ATAT1"    
 [103] "ATM"       "ATP1A1"    "ATP1A4"    "ATP2B2"    "ATP2B4"    "ATP7A"    
 [109] "ATP8B3"    "ATR"       "ATRX"      "AURKA"     "AURKC"     "AVP"      
 [115] "AVPR1A"    "AXL"       "AZI2"      "AZIN2"     "B4GALNT1"  "B4GALT1"  
 [121] "BACH1"     "BAD"       "BAG6"      "BAK1"      "BAP1"      "BASP1"    
 [127] "BAX"       "BBS2"      "BBS4"      "BCAP31"    "BCL2"      "BCL2L1"   
 [133] "BCL2L10"   "BCL2L11"   "BCL2L2"    "BCL6"      "BIK"       "BIRC2"    
 [139] "BIRC3"     "BIRC6"     "BMP15"     "BMP4"      "BMP5"      "BMP6"     
 [145] "BMP7"      "BMPR1A"    "BMPR1B"    "BMPR2"     "BNC1"      "BOK"      
 [151] "BOLL"      "BPTF"      "BPY2"      "BRCA2"     "BRD2"      "BRDT"     
 [157] "BRF1"      "BRINP1"    "BRIP1"     "BRME1"     "BSG"       "BTBD18"   
 [163] "BTG1"      "BUB1"      "BUB1B"     "BUB3"      "C11orf80"  "C14orf39" 
 [169] "C16orf92"  "C1QBP"     "C3"        "C3orf62"   "C9orf24"   "CABS1"    
 [175] "CABYR"     "CACNA1H"   "CACNA1I"   "CAD"       "CADM1"     "CALCA"    
 [181] "CALR"      "CALR3"     "CAPN2"     "CASP2"     "CASP3"     "CASP8"    
 [187] "CATSPER1"  "CATSPER2"  "CATSPER3"  "CATSPERB"  "CATSPERD"  "CATSPERE" 
 [193] "CATSPERG"  "CBL"       "CBX2"      "CBX7"      "CCDC134"   "CCDC136"  
 [199] "CCDC182"   "CCDC39"    "CCDC40"    "CCDC42"    "CCDC63"    "CCDC87"   
 [205] "CCIN"      "CCN1"      "CCNA1"     "CCNB1IP1"  "CCNB2"     "CCNB3"    
 [211] "CCND1"     "CCNE1"     "CCNE2"     "CCNF"      "CCNI"      "CCNYL1"   
 [217] "CCR6"      "CCT2"      "CCT3"      "CCT4"      "CCT5"      "CCT7"     
 [223] "CCT8"      "CD38"      "CD46"      "CD9"       "CDC20"     "CDC25A"   
 [229] "CDC25B"    "CDC25C"    "CDH1"      "CDK1"      "CDK16"     "CDK2"     
 [235] "CDKL2"     "CDKN1B"    "CDKN1C"    "CDX2"      "CDX4"      "CDY1"     
 [241] "CDYL"      "CEBPA"     "CEBPB"     "CECR2"     "CELF1"     "CELF3"    
 [247] "CELF4"     "CENPC"     "CENPI"     "CENPS"     "CENPX"     "CEP131"   
 [253] "CEP57"     "CETN2"     "CFAP157"   "CFAP206"   "CFAP251"   "CFAP43"   
 [259] "CFAP44"    "CFAP45"    "CFAP47"    "CFAP52"    "CFAP54"    "CFAP58"   
 [265] "CFAP65"    "CFAP69"    "CFAP91"    "CFAP97"    "CFAP97D1"  "CFTR"     
 [271] "CGB3

Now, we already had GO terms with genes. However, we still do not know the meaning of GO terms related to biological process. We can use `GO.db` database to get a set of annotation maps describing the entire Gene Ontology assembled using data from GO. We can use the following code to install the `GO.db` R package.

In [12]:
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("GO.db", update = F))
})
library(GO.db)

Then, we can use the following command to obtain the GO terms description.

In [13]:
# Getting the name of each GO term
terms <- names(allGO)
# Getting the description of each GO term
descriptions <-lapply(Term(terms), `[[`, 1)

In order to perform enrichment analysis in later module, we need to save the GO terms and genesets to the standard output. One commonly used format is Gene Matrix Transposed file format *(\*.gmt)*. The GMT file format is a tab delimited file format that describes gene sets. In the GMT format, each row represents a gene set; in the GMX format, each column represents a gene set. Here, we can save GO terms and genesets to the *\*.gmt* using the following function:

In [14]:
# A function to save the GO terms with geneset to the local repository
writeGMT <- function(genesets, descriptions, outfile) {

  if (file.exists(outfile)) {
    file.remove(outfile)
  }
  for (gs in names(genesets)) {
    write(c(gs, gsub("\t", " ", descriptions[[gs]]), genesets[[gs]]), file=outfile, sep="\t", append=TRUE, ncolumns=length(genesets[[gs]]) + 2)
  }
}
outfile <- "./data/GO_terms.gmt"
writeGMT(allGO, descriptions, outfile)

:::{Saving to Google Cloud Bucket}
Run the following command in the Vertex AI Workbench terminal to save the result to Google Cloud Bucket:

gsutil cp ./data/GO_terms.gmt gs://cpa-output
:::

## Kyoto Encyclopedia of Genes and Genomes (KEGG)
### Overview
KEGG is a collection of databases dealing with genomes, biological pathways, diseases, drugs, and chemical substances. KEGG is utilized for bioinformatics research and education, including data analysis in genomics, metagenomics, metabolomics and other omics studies, modeling and simulation in systems biology, and translational research in drug development. The KEGG database project was initiated in 1995 by Minoru Kanehisa, professor at the Institute for Chemical Research, Kyoto University, under the then ongoing Japanese Human Genome Program. Foreseeing the need for a computerized resource that can be used for biological interpretation of genome sequence data, he started developing the KEGG PATHWAY database. It is a collection of manually drawn KEGG pathway maps representing experimental knowledge on metabolism and various other functions of the cell and the organism. Each pathway map contains a network of molecular interactions and reactions and is designed to link genes in the genome to gene products (mostly proteins) in the pathway. This has enabled the analysis called KEGG pathway mapping, whereby the gene content in the genome is compared with the KEGG PATHWAY database to examine which pathways and associated functions are likely to be encoded in the genome. KEGG is a "computer representation" of the biological system. It integrates building blocks and wiring diagrams of the system—more specifically, genetic building blocks of genes and proteins, chemical building blocks of small molecules and reactions, and wiring diagrams of molecular interaction and reaction networks. The illustrative structure of KEGG is presented as figure below.
![](./images/Module3/KEGG.jpg)


### Retrieving pathways from KEGG databases
In this section, we will retrieve pathways and related gensets from the KEGG database using R command line. Here we will use `KEGGREST` R package that provides a client interface to the KEGG REST server. `KEGGREST` can be installed from the Bioconductor using following command.

In [1]:
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("KEGGREST", update = F))
})
suppressPackageStartupMessages({
  library(KEGGREST)
})

In [15]:
library(KEGGREST)

KEGG exposes a number of databases. To get an idea of what is available, run `listDatabases()`:

In [16]:
KEGGREST::listDatabases()

[1] "pathway"  "brite"    "module"   "ko"       "genome"   "vg"      
 [7] "ag"       "compound" "glycan"   "reaction" "rclass"   "enzyme"  
[13] "disease"  "drug"     "dgroup"   "environ"  "genes"    "ligand"  
[19] "kegg"

We can use these databases in further queries. Note that in many cases you can also use a three-letter KEGG organism code or a “T number” (genome identifier) in the same place you would use one of these database names.

We can obtain the list of organisms available in KEGG with the `keggList()` function:

In [17]:
organism <- keggList("organism")

In [18]:
print(paste0("KEGG supports ",dim(organism)[1]," organisms"))

[1] "KEGG supports 8455 organisms"


To view the supported organism we can use the following command:

In [19]:
# View several supported organism
head(organism)

T.number,organism,species,phylogeny
T01001,hsa,Homo sapiens (human),Eukaryotes;Animals;Vertebrates;Mammals
T01005,ptr,Pan troglodytes (chimpanzee),Eukaryotes;Animals;Vertebrates;Mammals
T02283,pps,Pan paniscus (bonobo),Eukaryotes;Animals;Vertebrates;Mammals
T02442,ggo,Gorilla gorilla gorilla (western lowland gorilla),Eukaryotes;Animals;Vertebrates;Mammals
T01416,pon,Pongo abelii (Sumatran orangutan),Eukaryotes;Animals;Vertebrates;Mammals
T03265,nle,Nomascus leucogenys (northern white-cheeked gibbon),Eukaryotes;Animals;Vertebrates;Mammals


In the Module 02, we performed DE analysis on human dataset. Therefore, we need to download pathways for human, the abbreviation of human pathway in KEGG is `hsa` and we can use `keggList` function to get the pathway list.

In [20]:
# Obtain the pathways belong to human
pathways.list <- keggList("pathway", "hsa")

The pathway list contains pathway description and pathway code in a single line of text. To see the first five pathways, we can use the following command:

In [35]:
# View the first five pathways
pathways.list[1:5]

path:hsa00010 
            "Glycolysis / Gluconeogenesis - Homo sapiens (human)" 
                                                    path:hsa00020 
               "Citrate cycle (TCA cycle) - Homo sapiens (human)" 
                                                    path:hsa00030 
               "Pentose phosphate pathway - Homo sapiens (human)" 
                                                    path:hsa00040 
"Pentose and glucuronate interconversions - Homo sapiens (human)" 
                                                    path:hsa00051 
         "Fructose and mannose metabolism - Homo sapiens (human)"

We can see that, in each line, the text in the quotation mark contain pathway information while the later part cotains pathway code leading by a prefix `path:`. To get  pathway codes from the pathway list, we can use the following command:

In [23]:
# Retrieve all the pathway IDs belong to human
pathway.codes <- sub("path:", "", names(pathways.list))
pathway.codes

[1] "hsa00010" "hsa00020" "hsa00030" "hsa00040" "hsa00051" "hsa00052"
  [7] "hsa00053" "hsa00061" "hsa00062" "hsa00071" "hsa00100" "hsa00120"
 [13] "hsa00130" "hsa00140" "hsa00190" "hsa00220" "hsa00230" "hsa00232"
 [19] "hsa00240" "hsa00250" "hsa00260" "hsa00270" "hsa00280" "hsa00290"
 [25] "hsa00310" "hsa00330" "hsa00340" "hsa00350" "hsa00360" "hsa00380"
 [31] "hsa00400" "hsa00410" "hsa00430" "hsa00440" "hsa00450" "hsa00470"
 [37] "hsa00480" "hsa00500" "hsa00510" "hsa00511" "hsa00512" "hsa00513"
 [43] "hsa00514" "hsa00515" "hsa00520" "hsa00524" "hsa00531" "hsa00532"
 [49] "hsa00533" "hsa00534" "hsa00561" "hsa00562" "hsa00563" "hsa00564"
 [55] "hsa00565" "hsa00590" "hsa00591" "hsa00592" "hsa00600" "hsa00601"
 [61] "hsa00603" "hsa00604" "hsa00620" "hsa00630" "hsa00640" "hsa00650"
 [67] "hsa00670" "hsa00730" "hsa00740" "hsa00750" "hsa00760" "hsa00770"
 [73] "hsa00780" "hsa00785" "hsa00790" "hsa00830" "hsa00860" "hsa00900"
 [79] "hsa00910" "hsa00920" "hsa00970" "hsa00980" "hsa00982" "hsa00983"
 [85] "hsa01040" "hsa01100" "hsa01200" "hsa01210" "hsa01212" "hsa01230"
 [91] "hsa01232" "hsa01240" "hsa01250" "hsa01521" "hsa01522" "hsa01523"
 [97] "hsa01524" "hsa02010" "hsa03008" "hsa03010" "hsa03013" "hsa03015"
[103] "hsa03018" "hsa03020" "hsa03022" "hsa03030" "hsa03040" "hsa03050"
[109] "hsa03060" "hsa03250" "hsa03260" "hsa03264" "hsa03265" "hsa03266"
[115] "hsa03320" "hsa03410" "hsa03420" "hsa03430" "hsa03440" "hsa03450"
[121] "hsa03460" "hsa04010" "hsa04012" "hsa04014" "hsa04015" "hsa04020"
[127] "hsa04022" "hsa04024" "hsa04060" "hsa04061" "hsa04062" "hsa04064"
[133] "hsa04066" "hsa04068" "hsa04070" "hsa04071" "hsa04072" "hsa04080"
[139] "hsa04110" "hsa04114" "hsa04115" "hsa04120" "hsa04122" "hsa04130"
[145] "hsa04136" "hsa04137" "hsa04140" "hsa04141" "hsa04142" "hsa04144"
[151] "hsa04145" "hsa04146" "hsa04150" "hsa04151" "hsa04152" "hsa04210"
[157] "hsa04211" "hsa04213" "hsa04215" "hsa04216" "hsa04217" "hsa04218"
[163] "hsa04260" "hsa04261" "hsa04270" "hsa04310" "hsa04330" "hsa04340"
[169] "hsa04350" "hsa04360" "hsa04370" "hsa04371" "hsa04380" "hsa04390"
[175] "hsa04392" "hsa04510" "hsa04512" "hsa04514" "hsa04520" "hsa04530"
[181] "hsa04540" "hsa04550" "hsa04610" "hsa04611" "hsa04612" "hsa04613"
[187] "hsa04614" "hsa04620" "hsa04621" "hsa04622" "hsa04623" "hsa04625"
[193] "hsa04630" "hsa04640" "hsa04650" "hsa04657" "hsa04658" "hsa04659"
[199] "hsa04660" "hsa04662" "hsa04664" "hsa04666" "hsa04668" "hsa04670"
[205] "hsa04672" "hsa04710" "hsa04713" "hsa04714" "hsa04720" "hsa04721"
[211] "hsa04722" "hsa04723" "hsa04724" "hsa04725" "hsa04726" "hsa04727"
[217] "hsa04728" "hsa04730" "hsa04740" "hsa04742" "hsa04744" "hsa04750"
[223] "hsa04810" "hsa04910" "hsa04911" "hsa04912" "hsa04913" "hsa04914"
[229] "hsa04915" "hsa04916" "hsa04917" "hsa04918" "hsa04919" "hsa04920"
[235] "hsa04921" "hsa04922" "hsa04923" "hsa04924" "hsa04925" "hsa04926"
[241] "hsa04927" "hsa04928" "hsa04929" "hsa04930" "hsa04931" "hsa04932"
[247] "hsa04933" "hsa04934" "hsa04935" "hsa04936" "hsa04940" "hsa04950"
[253] "hsa04960" "hsa04961" "hsa04962" "hsa04964" "hsa04966" "hsa04970"
[259] "hsa04971" "hsa04972" "hsa04973" "hsa04974" "hsa04975" "hsa04976"
[265] "hsa04977" "hsa04978" "hsa04979" "hsa05010" "hsa05012" "hsa05014"
[271] "hsa05016" "hsa05017" "hsa05020" "hsa05022" "hsa05030" "hsa05031"
[277] "hsa05032" "hsa05033" "hsa05034" "hsa05100" "hsa05110" "hsa05120"
[283] "hsa05130" "hsa05131" "hsa05132" "hsa05133" "hsa05134" "hsa05135"
[289] "hsa05140" "hsa05142" "hsa05143" "hsa05144" "hsa05145" "hsa05146"
[295] "hsa05150" "hsa05152" "hsa05160" "hsa05161" "hsa05162" "hsa05163"
[301] "hsa05164" "hsa05165" "hsa05166" "hsa05167" "hsa05168" "hsa05169"
[307] "hsa05170" "hsa05171" "hsa05200" "hsa05202" "hsa05203" "hsa05204"
[313] "hsa05205" "hsa05206" "hsa05207" "hsa05208" "hsa05210" "hsa05211"
[319] "hsa05212" "hsa05213" "hsa05214" "hsa05215" "hsa05216" "hsa05217"
[325] "hsa05218" "hsa05219" "hsa05220" "hsa05221" "hsa05222" "hsa05223"
[331] "hsa05224" "hsa05225" "hsa05226" "hs

We can use the following command to check how many pathways are available for human

In [24]:
print(paste0("Number of available pathways for human are: ", length(pathway.codes)))

[1] "Number of available pathways for human are: 351"


The following code will help to get list of genes and pathway's description for all pathways available in human.

In [25]:
# Function to get all the gene names for each pathway
genes.by.pathway <- sapply(pathway.codes,
                           function(pwid){
                             pw <- keggGet(pwid)
                             if (is.null(pw[[1]]$GENE)) return(NA)
                             pw2 <- pw[[1]]$GENE[c(FALSE,TRUE)]
                             pw2 <- unlist(lapply(strsplit(pw2, split = ";", fixed = T), function(x)x[1]))
                             return(pw2)
                           }
)
# Function to get description for each pathway
description.by.pathway <- sapply(pathway.codes,
                                 function(pwid){
                                   pw <- keggGet(pwid)
                                   if (is.null(pw[[1]]$NAME)) return(NA)
                                   pw2 <- pw[[1]]$NAME
                                   return(pw2)
                                 }
)
# Convert the pathway description to a list
description.by.pathway <- as.list(description.by.pathway)

We can view the first five pathways with their genesets using the following command

Then we can save the output to *\*.gmt* file using the following commands

In [26]:
# View the first five pathway with the genesets
genes.by.pathway[1:5]

$hsa00010
 [1] "HK3"     "HK1"     "HK2"     "HKDC1"   "GCK"     "GPI"     "PFKM"   
 [8] "PFKP"    "PFKL"    "FBP1"    "FBP2"    "ALDOC"   "ALDOA"   "ALDOB"  
[15] "TPI1"    "GAPDH"   "GAPDHS"  "PGK2"    "PGK1"    "PGAM1"   "PGAM2"  
[22] "PGAM4"   "ENO3"    "ENO2"    "ENO1"    "ENO4"    "PKM"     "PKLR"   
[29] "PDHA2"   "PDHA1"   "PDHB"    "DLAT"    "DLD"     "LDHAL6A" "LDHAL6B"
[36] "LDHA"    "LDHB"    "LDHC"    "ADH1A"   "ADH1B"   "ADH1C"   "ADH7"   
[43] "ADH4"    "ADH5"    "ADH6"    "AKR1A1"  "ALDH2"   "ALDH3A2" "ALDH1B1"
[50] "ALDH7A1" "ALDH9A1" "ALDH3B1" "ALDH3B2" "ALDH3A1" "ACSS1"   "ACSS2"  
[57] "GALM"    "PGM1"    "PGM2"    "G6PC1"   "G6PC2"   "G6PC3"   "ADPGK"  
[64] "BPGM"    "MINPP1"  "PCK1"    "PCK2"   

$hsa00020
 [1] "CS"     "ACLY"   "ACO2"   "ACO1"   "IDH1"   "IDH2"   "IDH3B"  "IDH3G" 
 [9] "IDH3A"  "OGDHL"  "OGDH"   "DLST"   "DLD"    "SUCLG1" "SUCLG2" "SUCLA2"
[17] "SDHA"   "SDHB"   "SDHC"   "SDHD"   "FH"     "MDH1"   "MDH2"   "PC"    
[25] "PCK1"   "PCK2"   "PDHA2"  "PDHA1"  "PDHB"   "DLAT"  

$hsa00030
 [1] "GPI"     "G6PD"    "PGLS"    "H6PD"    "PGD"     "RPE"     "RPEL1"  
 [8] "TKT"     "TKTL2"   "TKTL1"   "TALDO1"  "RPIA"    "DERA"    "RBKS"   
[15] "PGM1"    "PGM2"    "PRPS1L1" "PRPS2"   "PRPS1"   "RGN"     "IDNK"   
[22] "GLYCTK"  "ALDOC"   "ALDOA"   "ALDOB"   "FBP1"    "FBP2"    "PFKM"   
[29] "PFKP"    "PFKL"   

$hsa00040
 [1] "GUSB"    "KL"      "UGT2A1"  "UGT2A3"  "UGT2B17" "UGT2B11" "UGT2B28"
 [8] "UGT1A6"  "UGT1A4"  "UGT1A1"  "UGT1A3"  "UGT2B10" "UGT1A9"  "UGT2B7" 
[15] "UGT1A10" "UGT1A8"  "UGT1A5"  "UGT2B15" "UGT1A7"  "UGT2B4"  "UGT2A2" 
[22] "UGDH"    "UGP2"    "AKR1A1"  "CRYL1"   "RPE"     "RPEL1"   "XYLB"   
[29] "AKR1B1"  "AKR1B10" "DCXR"    "SORD"    "DHDH"    "FGGY"    "CRPPA"  

$hsa00051
 [1] "MPI"     "PMM2"    "PMM1"    "GMPPB"   "GMPPA"   "GMDS"    "GFUS"   
 [8] "FPGT"    "FCSK"    "ENOSF1"  "HK3"     "HK1"     "HK2"     "HKDC1"  
[15] "PFKM"    "PFKP"    "PFKL"    "FBP1"    "FBP2"    "PFKFB1"  "PFKFB2" 
[22] "PFKFB3"  "PFKFB4"  "TIGAR"   "KHK"     "SORD"    "AKR1B1"  "AKR1B10"
[29] "ALDOC"   "ALDOA"   "ALDOB"   "TPI1"    "TKFC"

Use the following command to see the description of the first five pathways

In [27]:
# View the description of the first five pathways
description.by.pathway[1:5]

$hsa00010
[1] "Glycolysis / Gluconeogenesis - Homo sapiens (human)"

$hsa00020
[1] "Citrate cycle (TCA cycle) - Homo sapiens (human)"

$hsa00030
[1] "Pentose phosphate pathway - Homo sapiens (human)"

$hsa00040
[1] "Pentose and glucuronate interconversions - Homo sapiens (human)"

$hsa00051
[1] "Fructose and mannose metabolism - Homo sapiens (human)"

In [28]:
# Saving the pathway information to the local repository
outfile <- "./data/KEGG_pathways.gmt"
writeGMT(genes.by.pathway, description.by.pathway, outfile)


:::{Saving to Google Cloud Bucket}
Run the following command in the Vertex AI Workbench terminal to save the result to Google Cloud Bucket:

gsutil cp ./data/KEGG_pathways.gmt gs://cpa-output
:::